In [1]:
# !pip install pymupdf4llm

In [3]:
!pip list | grep pymupdf4llm

pymupdf4llm                  0.0.24


In [1]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("/home/cristianomendieta/estudos/chat-with-pdf/case_files/LB5001.pdf")

In [2]:
print(md_text)

**AC & DC Motor Installation & Maintenance Instructions**
**Handling**
The weight of the motor and shipping container will vary. Use correct material handling
equipment to avoid injury.

**Receiving**
Inspect the motor for damage before accepting it. The Motor shaft should rotate freely with
no rubs. Report any damage immediately to the commercial carrier that delivered your
motor.

**Safety Notice**
Only qualified personnel trained in the safe installation and operation of this equipment
should install this motor. When improperly installed or used, rotating equipment can
cause serious or fatal injury. Equipment must be installed in accordance with the National
Electrical Code (NEC), local codes and NEMA MG2 Safety Standards for Construction and
Guide for Selection, Installation and Use of Electric Motors and Generators. Observe the
following guidelines:
1. When eyebolts are provided, they must be fully tightened and are intended to lift the
motor and its included accessories only.
2. 

In [3]:
import pprint

pprint.pp(md_text)

('**AC & DC Motor Installation & Maintenance Instructions**\n'
 '**Handling**\n'
 'The weight of the motor and shipping container will vary. Use correct '
 'material handling\n'
 'equipment to avoid injury.\n'
 '\n'
 '**Receiving**\n'
 'Inspect the motor for damage before accepting it. The Motor shaft should '
 'rotate freely with\n'
 'no rubs. Report any damage immediately to the commercial carrier that '
 'delivered your\n'
 'motor.\n'
 '\n'
 '**Safety Notice**\n'
 'Only qualified personnel trained in the safe installation and operation of '
 'this equipment\n'
 'should install this motor. When improperly installed or used, rotating '
 'equipment can\n'
 'cause serious or fatal injury. Equipment must be installed in accordance '
 'with the National\n'
 'Electrical Code (NEC), local codes and NEMA MG2 Safety Standards for '
 'Construction and\n'
 'Guide for Selection, Installation and Use of Electric Motors and Generators. '
 'Observe the\n'
 'following guidelines:\n'
 '1. When eyebol

In [12]:
md_text = pymupdf4llm.to_markdown(doc="/home/cristianomendieta/estudos/chat-with-pdf/case_files/WEG-motores-eletricos-guia-de-especificacao-50032749-brochure-portuguese-web.pdf",
                                      write_images=True,
                                      image_path="/home/cristianomendieta/estudos/chat-with-pdf/case_files/images-2")

In [13]:
import pprint

pprint.pp(md_text)

('# **GUIA DE ESPECIFICAÇÃO** **MOTORES ELÉTRICOS**\n'
 '\n'
 '\n'
 '![](/home/cristianomendieta/estudos/chat-with-pdf/case_files/images-2/WEG-motores-eletricos-guia-de-especificacao-50032749-brochure-portuguese-web.pdf-0-4.png)\n'
 '\n'
 '![](/home/cristianomendieta/estudos/chat-with-pdf/case_files/images-2/WEG-motores-eletricos-guia-de-especificacao-50032749-brochure-portuguese-web.pdf-0-5.png)\n'
 'www.weg.net\n'
 '\n'
 '2 Especificação do Motor Elétrico\n'
 '\n'
 '### i\n'
 '\n'
 'i\n'
 '\n'
 '\n'
 'www.weg.net\n'
 '### Especifcação i de Motores Elétricos\n'
 '##### Onde quer que haja progresso, a presença do motor elétrico é '
 'imprescindível. Desempenhando um importante papel para a sociedade, os '
 'motores são o coração das máquinas modernas, por essa razão é necessário '
 'conhecer seus princípios fundamentais de funcionamento, desde a construção '
 'até as aplicações. O Guia de Especificação de Motores Elétricos WEG auxilia '
 'de maneira simples e objetiva aqueles que compr